### Azure Speech-to-Text API Regression Test Notebook

Azure Speech to Text service transcribes spoken audio into written text with support for batch processing.

#### Key Features

- **Batch Transcription**: Converts spoken audio into text asynchronously, ideal for live apps or processing recordings.
- **Punctuation, Diarization & Translation**: Adds automatic punctuation, speaker diarization (who said what), and real-time speech translation.

#### Overview

1. **Setup and Configuration**:
    - Install necessary packages and import required modules.
    - Load environment variables and configure API keys.

By following these steps and utilizing the provided code snippets below, you can effectively manage and deploy Azure Speech Service to enhance your applications with AI-driven capabilities. For more detailed information, refer to the [Microsoft Learn documentation](http  s://learn.microsoft.com/en-us/azure/cognitive-services/openai/).

### Notebook Details
The following Python notebook will guide you through using the Azure Speech To Text APIs.

#### Assumptions/Prerequisites
- You have an API key with access to the Azure Speech To Text API in APIM.
- You have a user configuration file that specifies your access to AI services.

### Install the required packages

In [ ]:
#%pip install requests
#%pip install azure-cognitiveservices-speech
#%pip install azure-storage-blob

In [ ]:
import requests
import json

### Load the configuration settings

In [ ]:
# Replace these with your actual values
from dotenv import dotenv_values, load_dotenv
import sys
sys.path.append('../common')

load_dotenv()

secrets = dotenv_values("secrets.env")
API_KEY = secrets.get("API_KEY")
APIM_NAME = secrets.get("APIM_NAME")
API_VERSION = "2024-11-15"
region_override = secrets.get("REGION_OVERRIDE")

HEADERS = {
    "Ocp-Apim-Subscription-Key": API_KEY,
    "Content-Type": "application/json",
    "x-kpmg-charge-code": "1",
    "x-kpmg-region-override": region_override
}

### Upload blob to storage account

In [ ]:
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions
from datetime import datetime, timedelta
import os

# Set variables
account_connection_string = secrets.get("AZURE_STORAGE_CONNECTION_STRING")
container_name = "reg-source"
local_file_path = "./data/whatstheweatherlike.wav"  # e.g., "./data/report.pdf"
blob_name = os.path.basename(local_file_path)  # Use file name as blob name

# Initialize BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(account_connection_string)   
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Upload the file
with open(local_file_path, "rb") as data:
    blob_client.upload_blob(data, overwrite=True)
print(f"Uploaded '{local_file_path}' to container '{container_name}' as blob '{blob_name}'.")

print("Storage account name:", blob_service_client.account_name)

### Submit transcription job

In [ ]:
url = f"https://{APIM_NAME}/genai/azure/speech/speechtotext/transcriptions:submit?api-version={API_VERSION}"

payload = {
  "contentUrls": [
        f"https://{blob_service_client.account_name}.blob.core.windows.net/data/whatstheweatherlike.wav"
    ],
  "locale": "en-US",
  "displayName": "My Transcription",
  "properties": {
    "timeToLiveHours": 24,
    "languageIdentification": {
      "candidateLocales": [
        "en-US",
        "de-DE",
        "es-ES"
      ]
    }
  }
} 

response = requests.post(url, headers=HEADERS, json=payload)
print(response.json())
data = response.json()  # Convert response to a Python dictionary

# Now extract the 'self' URL
self_url = data.get('self', '')

# Find the part after "transcriptions/"
start = self_url.find('transcriptions/') + len('transcriptions/')
end = self_url.find('?', start)  # Find where the ID ends (before query params)

transcription_id = self_url[start:end]

print("Transcription Job Id :- ", transcription_id)

assert response.status_code == 200, f"Batch transcription job not created successfully with ID:{transcription_id}"


### Get transcriptions from given Job Id

In [ ]:
url = f"https://{APIM_NAME}/genai/azure/speech/speechtotext/transcriptions/{transcription_id}?api-version={API_VERSION}"
print(transcription_id)
HEADERS = {
    "Ocp-Apim-Subscription-Key": API_KEY,
    "x-kpmg-charge-code": "1",
    "x-kpmg-region-override": region_override
}

response = requests.get(url, headers=HEADERS)

assert response.status_code == 200, f"Get files by Job id got failed"
result = response.json()
print(response.json())
status = result.get('status')
print("Status:", status)

### Get all files associated with given transcription job id

In [ ]:
import re
import time

# Polling loop
while True:
    response = requests.get(url, headers=HEADERS)
    data = response.json()
    status = data.get("status")

    print(f"Current status: {status}")

    if status != "Running" and status != "NotStarted":
        break

    time.sleep(10)  # Wait 10 seconds before polling again

url = f"https://{APIM_NAME}/genai/azure/speech/speechtotext/transcriptions/{transcription_id}/files?api-version={API_VERSION}"

if status == "Succeeded":
    response = requests.get(url, headers=HEADERS)
    print(response.json())
    assert response.status_code == 200, "Get all files associated with given transcription job id got failed"
    

    # Extract the URL from the JSON
    data = response.json()
    self_url = data['values'][0]['self']

    # Use regex to extract the file ID
    match = re.search(r'files/([a-f0-9-]+)', self_url)
    file_id = match.group(1) if match else None
    print('Fileid : ', file_id)
else:
    file_id = ""
    print(f"Job ID- {transcription_id} is in {status} status.")

### Get particular file for given job id

In [ ]:
if file_id != "" :
    url = f"https://{APIM_NAME}/genai/azure/speech/speechtotext/transcriptions/{transcription_id}/files/{file_id}?api-version={API_VERSION}"
    print(transcription_id)
    print(file_id)


    response = requests.get(url, headers=HEADERS)

    assert response.status_code == 200, "Get particular file associated with given transcription job id got failed"
    print(response.json())
else :
    print("File Id is empty")

### Synchronous transcription of an audio file

In [ ]:
url = f"https://{APIM_NAME}/genai/azure/speech/speechtotext/transcriptions:transcribe?api-version={API_VERSION}"

# Constants
SPEECH_ENDPOINT = url
audio_file_path = "./data/whatstheweatherlike.wav"  # Local path to your audio file

# Definition JSON
definition = {
    "locales": ["en-US"]
}

# Open the audio file and send the request inside the block
with open(audio_file_path, 'rb') as audio_file:
    files = {
        'audio': (audio_file_path, audio_file, 'audio/wav'),
        'definition': (None, json.dumps(definition), 'application/json')
    }
    
    # Now the file is still open here!
    response = requests.post(url, headers=HEADERS, files=files)
 
# Pretty print the response
print(response.json())


### Get list of all locales available for offline transcription

In [ ]:
url = f"https://{APIM_NAME}/genai/azure/speech/speechtotext/transcriptions/locales?api-version={API_VERSION}"

response = requests.get(url, headers=HEADERS)

assert response.status_code == 200, "Get all locales failed"
print(response.json())